<a href="https://colab.research.google.com/github/djili/data-processing/blob/main/movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse de données  de films

Cet exercice consiste à lire un fichier CSV (lien)contenant des informations sur des films, à effectuer une série de transformations et d'actions, tout en intégrant l'utilisation d'accumulateurs pour suivre des métriques globales et des variables broadcast pour distribuer des données statiques aux nœuds de travail.

1.   Lire un fichier CSV contenant des informations sur des films(nous utiliserons le RDD du dataframe df.rdd);
2.   Utiliser des accumulateurs pour suivre le nombre total de lignes et le nombre de lignes valides;
3.   Utiliser des variables broadcast pour distribuer une liste de genres de films à surveiller;
4.   Calculer le nombre total de films pour chaque genre surveillé;
5.   Identifier les années avec le plus grand nombre de films pour chaque genre surveillé;

In [1]:
from pyspark.sql import SparkSession
import json

In [5]:
# Créer et recuperation de la session Spark
spark = SparkSession.builder.appName("Analyse Films").getOrCreate()
spark

In [7]:
# Lire le fichier CSV avec en-têtes
df = spark.read.option("header", True).csv("/content/movies_metadata.csv")
df.show()

+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------+--------------------+-----------------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|  status|             tagline|               title|   video|        vote_average|       vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+-----

In [10]:
#Récupérer le RDD depuis le DataFrame
rdd = df.rdd
rdd.take(1)

[Row(adult='False', belongs_to_collection="{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}", budget='30000000', genres="[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]", homepage='http://toystory.disney.com/toy-story', id='862', imdb_id='tt0114709', original_language='en', original_title='Toy Story', overview="Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.", popularity='21.946943', poster_path='/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', production_companies="[{'name': 'Pixar Animation Studios', 'id': 3}]", production_countries="[{'iso_3166_1': 'US', 'name': 'United States of America'}]", relea

In [13]:
# Creation et initialisation d'accumulateur a 0 pour compter le nombre de ligne et le npmbre de lignes totals
total_lines = spark.sparkContext.accumulator(0)
valid_lines = spark.sparkContext.accumulator(0)
total_lines, valid_lines

(Accumulator<id=4, value=0>, Accumulator<id=5, value=0>)

In [16]:
# Définir les genres à surveiller et les diffuser avec broadcast
genres_surveilles = ["Drama", "Comedy", "Action", "Thriller"]
broadcast_genres = spark.sparkContext.broadcast(genres_surveilles)


In [18]:
def traiter_ligne(row):
    global total_lines, valid_lines
    total_lines += 1
    try:
        genres_col = row['genres']
        release_date = row['release_date']
        if not genres_col or not release_date:
            return []
        # Extraire l'année
        year = release_date[:4]
        # Nettoyage et parsing du champ genres
        genres_json = json.loads(genres_col.replace("'", "\""))
        genres_extraits = [g['name'] for g in genres_json if 'name' in g]
        # Filtrer selon les genres surveillés
        genres_valides = [g for g in genres_extraits if g in broadcast_genres.value]
        if genres_valides and year.isdigit():
            valid_lines += 1
            return [(g, year) for g in genres_valides]
        else:
            return []
    except:
        return []

In [20]:
# Appliquer la fonction sur le RDD grace a flatMap
genre_annee_rdd = rdd.flatMap(traiter_ligne)
genre_annee_rdd.take(5)

[('Comedy', '1995'),
 ('Comedy', '1995'),
 ('Comedy', '1995'),
 ('Action', '1995'),
 ('Drama', '1995')]

In [23]:
# Compter le nombre total de films par genre surveillé
genre_count = genre_annee_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)
print(genre_count)

PythonRDD[55] at RDD at PythonRDD.scala:53


In [24]:
# Compter le nombre de films par (genre, année)
genre_year_count = genre_annee_rdd.map(lambda x: ((x[0], x[1]), 1)).reduceByKey(lambda a, b: a + b)
print(genre_year_count)

PythonRDD[60] at RDD at PythonRDD.scala:53


In [25]:
# Trouver l'année avec le plus grand nombre de films pour chaque genre
max_year_by_genre = genre_year_count.map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey().mapValues(lambda values: max(values, key=lambda v: v[1]))
print(max_year_by_genre)

PythonRDD[65] at RDD at PythonRDD.scala:53


In [26]:
# Afficher les résultats
print("Nombre total de lignes :", total_lines.value)
print("Nombre de lignes valides :", valid_lines.value)

print("\n Nombre total de films par genre surveillé :")
for genre, count in genre_count.collect():
    print(f"{genre} : {count}")

print("\n Année avec le plus de films pour chaque genre surveillé :")
for genre, (year, count) in max_year_by_genre.collect():
    print(f"{genre} : {year} ({count} films)")

Nombre total de lignes : 39819
Nombre de lignes valides : 28342

 Nombre total de films par genre surveillé :
Comedy : 10399
Action : 5455
Drama : 16778
Thriller : 6470

 Année avec le plus de films pour chaque genre surveillé :
Comedy : 2014 (441 films)
Action : 2011 (216 films)
Drama : 2014 (675 films)
Thriller : 2014 (313 films)
